# Running and analysing an M/M/1 queue with Ciw

### Import Ciw

In [1]:
import ciw

(ensure latest version of Ciw)

In [2]:
assert ciw.__version__ == '1.1.5'

### First define the and create a Network object

In [3]:
N = ciw.create_network(
    Arrival_distributions=[['Exponential', 3.0]],
    Service_distributions=[['Exponential', 5.0]],
    Number_of_servers=[1]
)

### Now create a Simulation object, and simulate for 1000 time units

In [4]:
ciw.seed(0)
Q = ciw.Simulation(N)

In [5]:
max_sim_time = 1000
Q.simulate_until_max_time(max_sim_time)

### Before analysing results, let's compute what the expected wait and expected queue length should be, according to standard queueing theory

In [6]:
traffic_intensity = 3.0 / 5.0

In [7]:
expected_queue_length = traffic_intensity / (1 - traffic_intensity)
print(expected_queue_length)

1.4999999999999998


In [8]:
expected_waiting_time = expected_queue_length / 5.0
print(expected_waiting_time)

0.29999999999999993


### Let's find the mean waiting time of the simulation

In [9]:
warmup = 200

In [10]:
recs = Q.get_all_records()

In [11]:
waits = [r.waiting_time for r in recs if r.arrival_date > warmup]
simulation_mean_wait = sum(waits)/len(waits)
print(simulation_mean_wait)

0.26793656400165083


### Now finding average queue length is more involved... 

In [12]:
def mean_queue_length(recs, max_sim_time, warm_up):
    arrival_time_stamps = [(r.arrival_date, r.queue_size_at_arrival + 1) for r in recs]
    exit_time_stamps = [(r.exit_date, r.queue_size_at_departure) for r in recs]
    time_stamps = arrival_time_stamps + exit_time_stamps + [(0, 0)]
    time_stamps.sort(key=lambda r: r[0])
    numerator = sum([time_stamps[i][1] * (time_stamps[i][0] - time_stamps[i - 1][0]) for i in range(1, len(time_stamps)) if time_stamps[i][0] > warm_up])
    mean_queue_length = numerator / (max_sim_time - warm_up)
    return mean_queue_length

In [13]:
simulation_mean_queue_length = mean_queue_length(recs, max_sim_time, warmup)
print(simulation_mean_queue_length)

1.6301674223750484


### Finally let's compare the analytical resutls to the simulation results

In [14]:
print("Analytical mean waiting time: " + str(expected_waiting_time))
print("Simulation mean waiting time: " + str(simulation_mean_wait))

Analytical mean waiting time: 0.29999999999999993
Simulation mean waiting time: 0.26793656400165083


In [15]:
print("Analytical mean queue length: " + str(expected_queue_length))
print("Simulation mean queue length: " + str(simulation_mean_queue_length))

Analytical mean queue length: 1.4999999999999998
Simulation mean queue length: 1.6301674223750484
